# Installation

In [1]:
%pip install --upgrade --quiet  langchain-pinecone langchain-openai langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 814.5/814.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.4/278.4 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [13]:
# !pip install llama-cpp-python
# !pip install huggingface_hub
# !pip install sentence-transformers
# !pip install ctransformers
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 2.6 MB/s eta 0:00:00


# Importing Library

In [11]:
from langchain.llms import CTransformers
from langchain.embeddings import HuggingFaceEmbeddings
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
from langchain_pinecone import PineconeVectorStore
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter

# Global Variables

In [9]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin"
pinecone_api_key = "6e6dbce3-5bfe-435b-aedf-ba88236eec69"
pinecone_api_env = "gcp-starter"
index_name = "aitesting"
host="https://aitesting-9pxzsyq.svc.gcp-starter.pinecone.io"
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# Download model files
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)
llm=CTransformers(model=model_path,
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

llm

CTransformers(client=<ctransformers.llm.LLM object at 0x784983e27d00>, model='/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', model_type='llama', config={'max_new_tokens': 512, 'temperature': 0.8})

#Loading Data and Preprocessing

In [15]:
loader = PyPDFLoader("/content/Medical_book.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
docs = text_splitter.split_documents(documents)
len(docs)

7020

# Pinecone data Insertion

In [19]:
query = "What are Allergies"
vec = embeddings.embed_query(query)
len(vec)

384

In [39]:
# Set the Pinecone API key
%env PINECONE_API_KEY="6e6dbce3-5bfe-435b-aedf-ba88236eec69"

# Set the Pinecone index name
%env PINECONE_INDEX_NAME="aitesting"


env: PINECONE_API_KEY="6e6dbce3-5bfe-435b-aedf-ba88236eec69"
env: PINECONE_INDEX_NAME="aitesting"


In [40]:
from langchain_pinecone import PineconeVectorStore
# from pinecone import Pinecone
# pc = Pinecone(api_key=pinecone_api_key)
docsearch = PineconeVectorStore.from_documents(docs, embeddings, index_name=index_name)

UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'X-Cloud-Trace-Context': 'd5c82809f791e721766687503a708e5c', 'Date': 'Thu, 11 Apr 2024 12:02:06 GMT', 'Content-Type': 'text/html', 'Server': 'Google Frontend', 'Content-Length': '15', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Invalid API Key


In [ ]:

chunk_embedding = embeddings_model.embed_query(query)
docsearch = pinecone.Index(name=index_name,api_key=pinecone_api_key, host="https://aitesting-9pxzsyq.svc.gcp-starter.pinecone.io")
xc = docsearch.query(vector=chunk_embedding, top_k=3, include_metadata=True)
print("Result", xc)